In [1]:
# allows to import own functions
import sys
import os
import re
root_project = re.findall(r'(^\S*TFM)', os.getcwd())[0]
sys.path.append(root_project)

In [12]:
import pandas as pd
from src.features.add_features import add_features
from src.utils.help_func import rename_indicators
import keras
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras import backend as K

pd.options.mode.chained_assignment = None  # default='warn'

In [13]:
df_raw = pd.read_csv('../../data/processed/simulation_results_REV4_test.csv')
df_raw

,i_country,R0,Tc,Tr,omega,ind_inf_1,ind_inf_2,ind_inf_sumgrad,ind_inf_p,ind_mort_1,ind_mort_2,ind_mort_3,ind_mort_sumgrad,n_closed,react_time,total_infected,total_deceased,total_recovered
0,MRT,6.957955,1.712730,11.917101,0.421474,5.200000e+00,8.642633e+00,2.391741e+01,0.000035,0.717949,4.460011,4.170287,9.5,11,20,1.128905e+06,9.414420e+05,1.873230e+05
1,CRI,10.071858,1.993278,20.076016,0.256798,8.853333e+01,2.085479e+02,1.121798e+03,0.000531,0.508660,1.089355,4.263092,240.0,18,16,2.215812e+09,1.856075e+09,3.600110e+08
2,DNK,13.220658,2.176956,28.780786,0.556694,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0,9,12,1.000000e+00,0.000000e+00,0.000000e+00
3,GRC,10.596798,0.878435,9.308599,0.953062,4.333333e+00,8.490222e+00,1.022128e+01,0.000012,0.823077,9.969037,7.228118,8.0,10,20,4.477377e+08,4.023809e+08,4.534857e+07
4,FIN,7.026326,2.165711,15.216989,0.433996,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0,9,18,1.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,PLW,15.000192,1.136469,17.047250,0.062791,1.058593e+05,9.130263e+04,3.299506e+06,176.353732,0.074554,0.049978,0.621937,118502.5,11,18,5.120920e+09,2.647732e+09,2.473577e+09
7996,GUM,15.249274,1.770462,26.998260,0.327660,9.250000e+01,2.244101e+02,1.052011e+03,0.016740,0.581622,1.391915,7.305558,288.0,13,10,3.000256e+09,2.695716e+09,3.047204e+08
7997,OMN,10.832344,1.294841,14.026168,0.455143,2.284667e+02,7.268619e+02,2.398521e+03,0.001419,0.589145,1.485245,5.108631,876.5,11,9,2.862542e+09,2.474888e+09,3.876690e+08
7998,NCL,13.829648,0.589161,8.147894,0.098775,8.396607e+06,1.930275e+06,1.220615e+08,886.778145,0.261722,0.367218,0.696197,7306416.5,14,7,4.966547e+09,2.214751e+09,2.751913e+09


In [14]:
df = add_features(df_raw)
df = rename_indicators(df)

/disks/hdd2T/Drive/TFM/conda_env/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [15]:
df = df.loc[df['total_deceased'] > 1 ]
print(df.shape)
df.dropna(inplace=True)
print(df.shape)

(5436, 39)
(5185, 39)


In [16]:
features = [
    'ind_1.1',
    'ind_1.2',
    'ind_1.3',
    'ind_1.4',
    'ind_2.1',
    'ind_2.2',
    'ind_2.3',
    'ind_2.4',
    'n_closed',
    'react_time',
    'betweenness_log',
    'degree_log',
    'closeness',
    'country_pop_log',
    'country_departures_log',
    'exposed_pop_log',
    'affected_population',
    ]

df = df[features]
df.head()

,ind_1.1,ind_1.2,ind_1.3,ind_1.4,ind_2.1,ind_2.2,ind_2.3,ind_2.4,n_closed,react_time,betweenness_log,degree_log,closeness,country_pop_log,country_departures_log,exposed_pop_log,affected_population
0,3.174607,-10.248013,1.648659,2.156707,2.251292,-0.331357,1.495151,1.427985,11,20,-9.989554,-2.763620,0.402904,15.297869,6.858565,19.156177,0.000125
1,7.022688,-7.540260,4.483379,5.340169,5.480639,-0.675976,0.085586,1.449995,18,16,-8.073405,-2.224624,0.408088,15.424837,7.991931,20.327708,0.246387
3,2.324472,-11.320802,1.466337,2.138915,2.079442,-0.194706,2.299484,1.977979,10,20,-6.129009,-0.971861,0.504545,16.188337,9.990124,20.978504,0.053415
5,17.296856,7.314873,19.338513,11.886407,15.410504,-1.486454,4.059960,-1.224325,0,14,-8.073405,-2.224624,0.408088,15.424837,7.991931,20.327708,0.227000
7,18.732431,8.647402,16.166950,14.393026,16.057403,-1.978746,-1.026244,-1.220580,18,7,-36.043653,-4.709530,0.383420,10.920745,7.980024,19.605983,0.011278


In [17]:
X = df.drop('affected_population', axis=1)
y = df['affected_population']

In [18]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
    
def coeff_determination(y_true, y_pred):
    """
    Implements the coefficient of determination to be used in a Keras model.

    Parameters
    ----------
    y_true : np.array
        Ground truth.
    y_pred : np.array
        Predictions.

    Returns
    -------
    float
        Coefficient of determination.

    """
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res / (SS_tot + K.epsilon()))

model = keras.models.load_model('models/neural_network.h5',
                                custom_objects={'root_mean_squared_error': root_mean_squared_error,
                                               'coeff_determination':coeff_determination})

In [19]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense_5 (Dense)              (None, 80)                1360      
_________________________________________________________________
dense_6 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_7 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_8 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 81        
Total params: 20,881
Trainable params: 20,881
Non-trainable params: 0
_________________________________________________________

In [ ]:
model.evaluate(X, y)